<a href="https://colab.research.google.com/github/hareeshpb/StockPredicter/blob/main/StockPredictionPyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pycaret-ts-alpha

     |████████████████████████████████| 6.3 MB 5.4 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 479 kB 5.0 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 113 kB 53.4 MB/s 
     |████████████████████████████████| 1.3 MB 23.7 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 16.9 MB 42 kB/s 
     |████████████████████████████████| 1.4 MB 15.9 MB/s 
     |████████████████████████████████| 1.7 MB 69.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 261 kB 64.1 MB/s 
     |████████████████████████████████| 2.0 MB 68.0 MB/s

In [3]:
import pandas as pd
from pycaret.datasets import get_data
import yfinance as yf
from pycaret.time_series import *
import plotly.express as px

In [4]:
def addfreq(data):
  data['Date']=data['Date'].astype(str)
  data['Date'] = pd.to_datetime(data['Date'])
  data = data.groupby('Date').sum()
  data = data.asfreq(freq ='D')
  data['Close'] = data['Close'].ffill()
  return data

In [5]:
def prediction_model_generate(data):
  setup(data, fh = 7, fold = 3, session_id = 123)
  best = compare_models()
  return finalize_model(best)


In [9]:
def predictor(model, days):
  prediction = predict_model(model, fh = days)
  return prediction.to_frame()

In [17]:
def complete_data(data, prediction):
  complete = data.append(prediction)
  complete.reset_index(level=0, inplace=True)
  complete.columns = ['Date','Close']
  complete = addfreq(complete)
  complete['MA50']=complete['Close'].rolling(50).mean()
  complete['MA200']=complete['Close'].rolling(200).mean()
  return complete

In [20]:
def predict(ticker, days):
  yfin = yf.Ticker(ticker)
  data = yfin.history(period="max")
  data = data.dropna()
  data = data[['Close']]
  data.reset_index(level=0, inplace=True)
  data['Date'] = pd.to_datetime(data['Date'])
  data = addfreq(data)
  model = prediction_model_generate(data)
  prediction = predictor(model, days)
  print(prediction)
  complete = complete_data(data, prediction)
  fig = px.line(complete)
  fig.show()


In [22]:
predict("ETH-USD",90)

,Model,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
auto_arima,Auto ARIMA,135.403,155.597,0.0304,0.0307,-1.7989,45.8533
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,151.848,173.014,0.034,0.0347,-2.4605,0.1033
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detren...,152.589,173.373,0.0342,0.0349,-2.4209,0.0567
ets,ETS,152.468,172.385,0.0341,0.0349,-2.438,0.5700
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,152.735,173.438,0.0342,0.035,-2.4237,0.0533
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,152.735,173.438,0.0342,0.035,-2.4237,0.0533
lar_cds_dt,Least Angular Regressor w/ Cond. Deseasonalize...,152.735,173.438,0.0342,0.035,-2.4237,0.0567
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,153.297,173.176,0.0344,0.0351,-2.4342,0.0700
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,153.308,173.177,0.0344,0.0351,-2.4345,0.0700
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasona...,156.512,176.966,0.035,0.0359,-2.5652,0.0600


                Close
2021-11-22  4285.0589
2021-11-23  4344.4988
2021-11-24  4302.7645
2021-11-25  4376.2585
2021-11-26  4391.9167
...               ...
2022-02-15  5372.6075
2022-02-16  5384.9424
2022-02-17  5397.2773
2022-02-18  5409.6121
2022-02-19  5421.9470

[90 rows x 1 columns]
